<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [1]:
import requests
import json
import pandas as pd

In [7]:
routes = json.load(open('../data/raw/moovit_routes.json', 'r'))

In [12]:
routes = pd.DataFrame(routes['routes'])

In [21]:
agencias = pd.DataFrame(json.load(open('../data/raw/moovit_agencias.json', 'r')))

In [15]:
empresas = pd.read_csv('../data/raw/empresas_moovit.csv', encoding='latin1')

In [25]:
sel_agencias = pd.merge(agencias, empresas, left_on='agencyName', right_on='empresas_moovit')

In [30]:
sel_routes = routes[routes['agencyId'].isin(sel_agencias['agencyId'])]

In [54]:
def get_trips(route_id):
    
    url = f"https://editor.moovit.com/web/routes/{route_id}/trips"

    headers = {
        'host': "editor.moovit.com",
        'connection': "keep-alive",
        'accept': "application/json, text/plain, */*",
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.86 Safari/537.36",
        'sec-gpc': "1",
        'sec-fetch-site': "same-origin",
        'sec-fetch-mode': "cors",
        'sec-fetch-dest': "empty",
        'referer': "https://editor.moovit.com/web/routes/19692",
        'accept-encoding': "gzip, deflate, br",
        'accept-language': "en-GB,en-US;q=0.9,en;q=0.8",
        'cookie': "cookieconsent_status=dismiss; EDITORSESSIONID=Bp5CKqRGp5ZxrOrER0DRKJmD.undefined; JSESSIONIDSSO=sgKs7crBsVSLH+lknszk7Gme",
        'cache-control': "no-cache",
        'postman-token': "cc57a4c1-77f6-1d65-b5d5-f0939ff0bca9"
        }
    response = requests.request("GET", url, headers=headers)
    
    if response.ok:
        json.dump(response.json(), open(f'../data/raw/moovit_trips/{route_id}.json', 'w'))
    else:
        print(f'FAILED: {route_id}')

In [49]:
import requests


def fecth_and_save_stops(trip_id):
    
    url = f"https://editor.moovit.com/web/trips/mobile-mappings/{trip_id}"

    headers = {
        'host': "editor.moovit.com",
        'connection': "keep-alive",
        'accept': "application/json, text/plain, */*",
        'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.86 Safari/537.36",
        'sec-gpc': "1",
        'sec-fetch-site': "same-origin",
        'sec-fetch-mode': "cors",
        'sec-fetch-dest': "empty",
        'referer': "https://editor.moovit.com/web/trips/31699",
        'accept-encoding': "gzip, deflate, br",
        'accept-language': "en-GB,en-US;q=0.9,en;q=0.8",
        'cookie': "cookieconsent_status=dismiss; EDITORSESSIONID=Bp5CKqRGp5ZxrOrER0DRKJmD.undefined; JSESSIONIDSSO=sgKs7crBsVSLH+lknszk7Gme",
        'cache-control': "no-cache",
        'postman-token': "c19ef460-5f4d-0936-1225-18569ee1f312"
        }

    response = requests.request("GET", url, headers=headers)
    
    if response.ok:
        json.dump(response.json(), open(f'../data/raw/moovit_stops/{trip_id}.json', 'w'))
    else:
        print(f'FAILED: {trip_id}')

In [52]:
sel_routes.head()

,routeColor,export,mappedTrips,trips,routeLongName,lastUpdate,routeTextColor,userEmail,routeId,agencyId,agencyFeedId,feedId,routeType,departuresPercentage,mappedPercentage,name,routeShortName,departureTrips,userName
242,#009bff,True,2,2,Silvestre - Castelo (BRS 1),1597885096000,#FFFFFF,6b8250244cb1fb7fea033557a658a944,19650,50008,None,None,3,100,100,Gabriel Petersen Gomes,006,2,Gabriel Petersen Gomes
243,#009bff,False,2,2,Silvestre - Central (via Largo das Neves / Lar...,1597889692000,#FFFFFF,6b8250244cb1fb7fea033557a658a944,795295,50008,None,None,3,100,100,Gabriel Petersen Gomes,006SN,2,Gabriel Petersen Gomes
245,#009bff,True,3,3,Silvestre - Central (BRS 1),1606484688000,#FFFFFF,6b8250244cb1fb7fea033557a658a944,19691,50008,None,None,3,100,100,Gabriel Petersen Gomes,007,3,Gabriel Petersen Gomes
255,#009bff,False,4,4,Bairro de Fátima - Central (BRS 1),1616259049000,#FFFFFF,6b8250244cb1fb7fea033557a658a944,19278,50008,None,None,3,100,100,Gabriel Petersen Gomes,010,4,Gabriel Petersen Gomes
260,#009bff,True,2,2,Paula Matos - Castelo (BRS 1),1597885197000,#FFFFFF,6b8250244cb1fb7fea033557a658a944,19314,50008,None,None,3,100,100,Gabriel Petersen Gomes,014,2,Gabriel Petersen Gomes


In [57]:
import time
import tqdm
for route_id in tqdm.tqdm(sel_routes['routeId'].unique()):
    get_trips(route_id)
    time.sleep(1)

100%|██████████| 508/508 [12:05<00:00,  1.43s/it]


In [58]:
from pathlib import Path

In [60]:
trips = [json.load(open(p)) for p in Path('../data/raw/moovit_trips/').glob('*')]

In [63]:
flatten = lambda t: [item for sublist in t for item in sublist]
trips = flatten(trips)

In [65]:
trips = pd.DataFrame(trips)

In [67]:
len(trips)

1168

In [70]:
for trip_id in tqdm.tqdm(trips['tripId'].unique()):
    fecth_and_save_stops(trip_id)
    time.sleep(0.5)

100%|██████████| 1168/1168 [18:27<00:00,  1.05it/s]


In [48]:
!ls ../data/raw/moovit_stops/

In [71]:
stops = [json.load(open(p)) for p in Path('../data/raw/moovit_stops/').glob('*')]

In [73]:
stops = flatten(stops)

In [97]:
fstops = flatten([s['content']['stops'] for s in stops])

In [98]:
fstops = pd.DataFrame(fstops)[['lat', 'lng', 'name']]

In [99]:
fstops['lat'] = fstops['lat'].apply(lambda x: round(x, 5))
fstops['lng'] = fstops['lng'].apply(lambda x: round(x, 5))

In [101]:
fstops = fstops.drop_duplicates()

In [102]:
fstops.to_csv('../data/output/moovit_paradas.csv')